In [1]:
import logging
logging.basicConfig(level=logging.DEBUG, format='{asctime} - {name} - {levelname} - {message}', style='{')
import numpy as np, matplotlib.pyplot as plt, pandas as pd
# For finding and loading multiple files
import os
# Use this to see how much memory the dataframes use
from sys import getsizeof
# To view keyspace.yaml
import yaml
from pathlib import Path
from linetimer import CodeTimer, linetimer

import pseudopeople as psp

from vivarium_research_prl.utils import sizemb, MappingViaAttributes, build_full_address
from vivarium_research_prl import alpha

!date
!whoami
!uname -a
!python --version
!pwd

Wed 23 Aug 2023 12:07:03 AM PDT
ndbs
Linux long-slurm-sarchive-p0008 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
Python 3.10.12
/mnt/share/code/ndbs/vivarium_research_prl/pseudopeople_testing


In [2]:
# logging.basicConfig(level=logging.DEBUG, format='{asctime} - {name} - {levelname} - {message}', style='{', force=True)

In [3]:
%load_ext autoreload
%autoreload 2

# Things to test

1. Do all the `generate_x` functions work when passed either a string or a Path object?
2. Do all the datasets have the right columns?
3. Do the generate functions work when `NO_NOISE` is passed? Do they run faster?
4. 

In [88]:
# Set up a logger
file_logger = logging.getLogger(__name__) # This gets a new logger for the current, __main__ module

# # Define a console handler with a specified format
# console_handler = logging.StreamHandler()
# console_handler.setLevel(logging.DEBUG)
# console_format = logging.Formatter('{name} - {levelname} - {message}', style='{')
# console_handler.setFormatter(console_format)

# Create and configure file log handler
file_handler = logging.FileHandler('data_generation.log')
file_handler.setLevel(logging.DEBUG)
file_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_format)

# Define a 

# Add handlers to the logger
# logger.addHandler(console_handler)
file_logger.addHandler(file_handler)

In [89]:
!ls *.log

data_generation.log


In [90]:
# File is empty
!head data_generation.log

In [70]:
alpha.logger.name

'vivarium_research_prl.alpha'

In [71]:
alpha.logger.parent

<RootLogger root (DEBUG)>

In [72]:
root_logger = logging.getLogger()
root_logger

<RootLogger root (DEBUG)>

# Find data

NEW USA:

`/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop
/results/release_02_yellow/full_data
/united_states_of_america/2023_07_28_08_33_09
/final_results/2023_08_16_09_58_54/pseudopeople_input_data_usa`

NEW RI:

`/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop
/results/release_02_yellow/full_data
/united_states_of_america/2023_07_28_08_33_09
/final_results/2023_08_16_09_58_54/states/pseudopeople_input_data_rhode_island`

In [4]:
project_dir = Path('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop')
full_data_dir = project_dir / 'results/release_02_yellow/full_data'
usa_model_dir = full_data_dir / 'united_states_of_america/2023_07_28_08_33_09'
usa_data_dir = usa_model_dir / 'final_results/2023_08_16_09_58_54/pseudopeople_input_data_usa'
ri_data_dir = usa_model_dir / 'final_results/2023_08_16_09_58_54/states/pseudopeople_input_data_rhode_island'

!ls -halt $ri_data_dir

total 228K
drwxrwsr-x 9 albrja IHME-Simulationscience 3.5K Aug 17 19:16 .
drwxrwsr-x 2 albrja IHME-Simulationscience    0 Aug 17 19:16 taxes_dependents
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 19:16 taxes_1040
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 16:39 taxes_w2_and_1099
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 14:20 women_infants_and_children
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 14:13 current_population_survey
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 14:08 american_community_survey
drwxrwsr-x 2 albrja IHME-Simulationscience 167K Aug 17 14:06 decennial_census
drwxrwsr-x 3 albrja IHME-Simulationscience  512 Aug 17 13:13 ..


# Try loading all sample data with updates to generate_datasets

In [5]:
data = alpha.generate_datasets()
data.keylist()

2023-08-23 00:07:05,618 - vivarium_research_prl.alpha - INFO - Code block 'generate_american_community_survey' took: 0.01875 m
2023-08-23 00:07:06,640 - vivarium_research_prl.alpha - INFO - Code block 'generate_current_population_survey' took: 0.01701 m
2023-08-23 00:07:09,196 - vivarium_research_prl.alpha - INFO - Code block 'generate_decennial_census' took: 0.04260 m
2023-08-23 00:07:11,243 - vivarium_research_prl.alpha - INFO - Code block 'generate_social_security' took: 0.03410 m
2023-08-23 00:07:17,331 - vivarium_research_prl.alpha - INFO - Code block 'generate_taxes_1040' took: 0.10146 m
2023-08-23 00:07:22,307 - vivarium_research_prl.alpha - INFO - Code block 'generate_taxes_w2_and_1099' took: 0.08293 m
2023-08-23 00:07:23,420 - vivarium_research_prl.alpha - INFO - Code block 'generate_women_infants_and_children' took: 0.01854 m


['american_community_survey',
 'current_population_survey',
 'decennial_census',
 'social_security',
 'taxes_1040',
 'taxes_w2_and_1099',
 'women_infants_and_children']

In [6]:
data.taxes_w2_and_1099

,simulant_id,household_id,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,...,employer_id,employer_name,employer_street_number,employer_street_name,employer_unit_number,employer_city,employer_state,employer_zipcode,tax_form,tax_year
0,0_4,0_8,Eric,R,Stark Lozano,1501,interlake ave n,NaN,NaN,Anytown,...,95,Pikes Creek Campground,e,ince dr,NaN,Anytown,US,00000,W2,2020
1,0_5,0_8,Erin,R,Stark Lozano,1501,interlake ave n,NaN,NaN,Anytown,...,29,Red's Dairy Queen,264-12,hallmont dr,NaN,Anytown,US,00000,W2,2020
2,0_5,0_8,Erin,R,Stark Lozano,1501,interlake ave n,NaN,NaN,Anytown,...,30,Warrensburg,NaN,ne 39th ave,NaN,Anytown,US,00000,W2,2020
3,0_5621,0_2289,Derick,W,Campusano Robledo,,,,14011.0,Anytown,...,46,Nashville City Properties,283,west id rd,NaN,Anytown,US,00000,W2,2020
4,0_5623,0_2289,Heather,J,Campusano Robledo,,,,14011.0,Anytown,...,83,Ecr Whipple Oliver Finley Shoe Sensation,4682,buck tl,NaN,Anytown,US,00000,W2,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,0_18936,0_7621,Jimmy,T,Diaz,512,w shadowbend ave,NaN,NaN,Anytown,...,23,Greenway Med Spa Services,631,regent ave,NaN,Anytown,US,00000,W2,2020
9968,0_18936,0_7621,Jimmy,NaN,Diaz,512,w shadowbend ave,NaN,NaN,Anytown,...,90,Northwell Health Center Inc,NaN,barge roa sw,NaN,Anytown,US,00000,W2,2020
9969,0_18937,0_7621,Kathleen,M,Diaz,512,w shadowbend ave,NaN,NaN,Anytown,...,1,Military,7930,moriah rd,NaN,Anytown,US,00000,1099,2020
9970,0_18937,0_7621,Kathleen,M,Diaz,512,w shadowbend ave,NaN,NaN,Anytown,...,105,Waffle House of Big Daddys Burgers Shakes and ...,2329,greenwood wy,NaN,Anytown,US,00000,NaN,2020


# Try generating W2 dataset with several different seeds to see if I can reproduce the bug I got earlier

When I first ran the above codeblock, the logger logged a pandas error trying to cast `NaN`, `NA`, or `inf` to `int` when generating the W2/1099 dataset. Specifically, it looked like this was happening then casting back to the original dtype after noising. I was able to reproduce the error several times, specifically with the W2/1099 dataset but not others. However, when I restarted my kernel, the error disappeard, and I hadn't recorded the stack trace, so I don't have a record of it. It's possible that the kernel was using code from a different branch but data from develop, and the problem was fixed when I restarted the kernel because the code and data were then synced.

The block below successfully generated W2/1099 data with 100 different seeds, whereas before restarting my kernel, I couldn't generate this dataset at all.

In [23]:
%%time
for seed in range(100):
    with CodeTimer(f"Generate W2&1099 with {seed=}", unit='s'):
        w2 = psp.generate_taxes_w2_and_1099(seed=seed)

Code block 'Generate W2&1099 with seed=0' took: 4.83423 s


Code block 'Generate W2&1099 with seed=1' took: 4.48398 s


Code block 'Generate W2&1099 with seed=2' took: 5.02672 s


Code block 'Generate W2&1099 with seed=3' took: 4.76207 s


Code block 'Generate W2&1099 with seed=4' took: 4.53564 s


...

Code block 'Generate W2&1099 with seed=95' took: 4.61041 s


Code block 'Generate W2&1099 with seed=96' took: 4.95983 s


Code block 'Generate W2&1099 with seed=97' took: 4.91383 s


Code block 'Generate W2&1099 with seed=98' took: 4.74007 s


Code block 'Generate W2&1099 with seed=99' took: 4.88933 s
CPU times: user 7min 50s, sys: 4.77 s, total: 7min 55s
Wall time: 7min 56s


In [24]:
w2.equals(data.taxes_w2_and_1099)

False

# Look at some of the sample datasets

In [25]:
data.american_community_survey

,simulant_id,household_id,survey_date,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity
0,0_12123,0_4898,2020-10-06,Jamie,N,Valdez,43,07/04/1977,820,cameron road,NaN,Anytown,US,00000,Household,Opp-sex spouse,Female,Asian
1,0_12124,0_4898,2020-10-06,Raymond,J,Valdez,16,10/22/2003,820,cameron road,NaN,Anytown,US,00000,Household,Biological child,Male,Asian
2,0_12125,0_4898,2020-10-06,Anastasia,S,Valdez,47,03/17/2003,820,cameron road,NaN,Anytown,US,00000,Household,Biological child,Female,Asian
3,0_20535,0_4898,2020-10-06,Ronin,K,Valdez,16,08/04/2020,820,cameron road,NaN,Anytown,US,00000,Household,Biological child,Male,Asian
4,0_4493,0_1830,2020-02-25,Mark,E,Marsland,65,03/11/1954,NaN,nichole dr,NaN,Anytown,US,00000,Household,Reference person,Male,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0_20027,0_3,2020-04-21,Bryce,J,Woo,21,08/17/1998,8203,west farwell avenue,NaN,Anytown,US,00000,College,Noninstitutionalized GQ pop,Male,Asian
67,0_20329,0_3,2020-04-21,Zachary,J,Walker,17,01/20/2003,8203,west farwell avenue,NaN,Anytown,US,00000,College,Noninstitutionalized GQ pop,Male,White
68,0_20337,0_3,2020-04-21,Ella,P,Houston,0,01/01/2020,8203,west farwell avenue,NaN,Anytown,US,00000,College,Noninstitutionalized GQ pop,Female,Black
69,0_2895,0_1179,2020-12-01,Justin,J,Nelson,30,09/13/1987,111-24,gonzalez dr,apt # 5002,Anytown,US,00000,Household,Reference person,Male,White


In [9]:
data.taxes_1040

,simulant_id,household_id,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,...,dependent_2_first_name,dependent_2_last_name,dependent_2_ssn,dependent_3_first_name,dependent_3_last_name,dependent_3_ssn,dependent_4_first_name,dependent_4_last_name,dependent_4_ssn,tax_year
0,0_5621,0_2289,Derick,W,Campusano Robledo,,,,14011.0,Anytown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
1,0_7251,0_2957,Patrick,E,Velazquez,927,broomfield ln,NaN,NaN,Anytown,...,Henry,Velazquez,743-21-4895,Caleb,Velazquez,161-89-7593,NaN,NaN,NaN,2020
2,0_11263,0_8694,David,J,Russ,2412,n buchanan st,NaN,NaN,Anytown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,0_21,0_14,Gianna,R,Sperr,13208,w warren ave,NaN,NaN,Anytown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,0_8354,0_3397,Jason,F,Etter,38297,hunters oak drive,NaN,NaN,Anytown,...,Alejandro,Etter,360-78-0427,NaN,NaN,NaN,NaN,NaN,NaN,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6513,0_12765,0_5140,Lucas,W,Garrett,1680,lakota pnte ln,NaN,NaN,Anytown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
6514,0_15865,0_6729,Nichelle,B,Bridge,1460,duncroft ln,NaN,NaN,Anytown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
6515,0_16673,0_6729,Annmarie,C,Woolley,1460,duncrpft ln,NaN,NaN,Anytown,...,Avi,Woolley,112-67-6411,NaN,NaN,NaN,NaN,NaN,NaN,2020
6516,0_16675,0_6729,Vincent,J,Son,1460,duncroft ln,NaN,NaN,Anytown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [10]:
data.decennial_census

,simulant_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year
0,0_2,0_7,Diana,P,Kelly,25,05/06/1994,5112,145th st,NaN,Anytown,US,00000,Household,Reference person,Female,White,2020
1,0_3,0_7,Anna,A,Kelly,25,09/29/1994,5112,145th st,NaN,Anytown,US,00000,Household,Other relative,Female,White,2020
2,0_923,0_8033,Gerald,R,AlJen,76,11/03/1943,1130,mallory ln,NaN,Anytown,US,00000,Household,Reference person,Male,Black,2020
3,0_2641,0_1066,Loretta,T,NaN,61,06/21/1958,32537,delacorte dr,NaN,Anytown,US,00000,Household,Reference person,Female,White,2020
4,0_2801,0_1138,Richard,R,Pinard,73,03/03/1947,950,caribou lane,NaN,Anytown,US,00000,Household,Reference person,Male,White,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,0_18969,0_7630,Patty,E,Palmisano,87,01/11/1933,1706,lincoln ave se,NaN,Anytown,US,00000,Household,Opp-sex spouse,Female,White,2020
10242,0_19008,0_8361,John,V,Skeeter,58,12/29/1961,7736,trophy dr,NaN,Anytown,US,00000,Household,Reference person,Male,Black,2020
10243,0_20165,0_7999,Kimberly,K,Suitt,65,04/05/1955,3506,oneill avenue,NaN,Anytown,US,00000,Carceral,Reference person,Female,White,2020
10244,0_19020,0_8130,Virginia,G,Hoover,93,10/02/1926,201,edinburgh rd,NaN,Anytown,US,NaN,Household,Reference person,Female,NaN,2020


# See if I can get a CodeTimer to work

Yes, it works as expected.

When I first tried it above with a logger, it didn't print the output. The problem was that the root logger was set to level `WARNING`, whereas the time info in the `alpha` module's logger is recorded at level `INFO`. I fixed this by calling `logging.basicConfig(level=logging.DEBUG)` at the beginning of the notebook instead of creating my own logger object in the notebook (which was named `__main__` instead of `root`), but I don't know if that's the "correct" solution.

In [11]:
with CodeTimer('Yak', unit='s'):
    census = psp.generate_decennial_census()

census

Code block 'Yak' took: 2.91809 s


,simulant_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year
0,0_2,0_7,Diana,P,Kelly,25,05/06/1994,5112,145th st,NaN,Anytown,US,00000,Household,Reference person,Female,White,2020
1,0_3,0_7,Anna,A,Kelly,25,09/29/1994,5112,145th st,NaN,Anytown,US,00000,Household,Other relative,Female,White,2020
2,0_923,0_8033,Gerald,R,AlJen,76,11/03/1943,1130,mallory ln,NaN,Anytown,US,00000,Household,Reference person,Male,Black,2020
3,0_2641,0_1066,Loretta,T,NaN,61,06/21/1958,32537,delacorte dr,NaN,Anytown,US,00000,Household,Reference person,Female,White,2020
4,0_2801,0_1138,Richard,R,Pinard,73,03/03/1947,950,caribou lane,NaN,Anytown,US,00000,Household,Reference person,Male,White,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,0_18969,0_7630,Patty,E,Palmisano,87,01/11/1933,1706,lincoln ave se,NaN,Anytown,US,00000,Household,Opp-sex spouse,Female,White,2020
10242,0_19008,0_8361,John,V,Skeeter,58,12/29/1961,7736,trophy dr,NaN,Anytown,US,00000,Household,Reference person,Male,Black,2020
10243,0_20165,0_7999,Kimberly,K,Suitt,65,04/05/1955,3506,oneill avenue,NaN,Anytown,US,00000,Carceral,Reference person,Female,White,2020
10244,0_19020,0_8130,Virginia,G,Hoover,93,10/02/1926,201,edinburgh rd,NaN,Anytown,US,NaN,Household,Reference person,Female,NaN,2020


# Test `generate_datasets` again after adding parameters to customize the CodeTimer

In [26]:
# Pass None for the logger function to print the time to stdout
data1 = alpha.generate_datasets(code_timer_logger_func=None)
data1.keylist()

Code block 'generate_american_community_survey' took: 0.01530 m


Code block 'generate_current_population_survey' took: 0.01243 m


Code block 'generate_decennial_census' took: 0.03991 m


Code block 'generate_social_security' took: 0.03237 m


Code block 'generate_taxes_1040' took: 0.09966 m


Code block 'generate_taxes_w2_and_1099' took: 0.07661 m


Code block 'generate_women_infants_and_children' took: 0.01762 m


['american_community_survey',
 'current_population_survey',
 'decennial_census',
 'social_security',
 'taxes_1040',
 'taxes_w2_and_1099',
 'women_infants_and_children']

In [27]:
data1.taxes_w2_and_1099.equals(data.taxes_w2_and_1099)

True

In [28]:
%%time
data2 = alpha.generate_datasets(code_timer_unit='s')
data2.keylist()

2023-08-23 00:36:43,631 - vivarium_research_prl.alpha - INFO - Code block 'generate_american_community_survey' took: 0.91341 s
2023-08-23 00:36:44,424 - vivarium_research_prl.alpha - INFO - Code block 'generate_current_population_survey' took: 0.79197 s
2023-08-23 00:36:47,116 - vivarium_research_prl.alpha - INFO - Code block 'generate_decennial_census' took: 2.69152 s
2023-08-23 00:36:49,052 - vivarium_research_prl.alpha - INFO - Code block 'generate_social_security' took: 1.93506 s
2023-08-23 00:36:54,948 - vivarium_research_prl.alpha - INFO - Code block 'generate_taxes_1040' took: 5.89578 s
2023-08-23 00:36:59,638 - vivarium_research_prl.alpha - INFO - Code block 'generate_taxes_w2_and_1099' took: 4.68951 s
2023-08-23 00:37:00,721 - vivarium_research_prl.alpha - INFO - Code block 'generate_women_infants_and_children' took: 1.08253 s


CPU times: user 17.8 s, sys: 160 ms, total: 17.9 s
Wall time: 18 s


['american_community_survey',
 'current_population_survey',
 'decennial_census',
 'social_security',
 'taxes_1040',
 'taxes_w2_and_1099',
 'women_infants_and_children']

# Load all Rhode Island data and time it

Darn, I should have logged to a file. There was too much output to fit into the display, so I lost the records of the runtime.

In [37]:
%%time
ri_data = alpha.generate_datasets(ri_data_dir)
ri_data.keylist()

Applying noise: 100%|██████████████████████████████████████████████| 14/14 [00:00<00:00, 30.80type/s]
                                                                                                     
Noising data:  51%|████████████████████████▌                       | 171/334 [01:37<01:35,  1.70it/s]

Applying noise:  86%|███████████████████████████████████████▍      | 12/14 [00:00<00:00, 36.22type/s]
2023-08-23 02:12:48,631 - vivarium_research_prl.alpha - INFO - Code block 'generate_american_community_survey' took: 3.26285 m
Applying noise:   0%|                                                       | 0/14 [00:00<?, ?type/s]
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [38]:
ri_data.keylist()

['american_community_survey',
 'current_population_survey',
 'decennial_census',
 'social_security',
 'taxes_1040',
 'taxes_w2_and_1099',
 'women_infants_and_children']

# See if there were any errors

Yes, because there is no SSA data for Rhode Island.

In [41]:
{k: type(v) for k, v in ri_data.items()}

{'american_community_survey': pandas.core.frame.DataFrame,
 'current_population_survey': pandas.core.frame.DataFrame,
 'decennial_census': pandas.core.frame.DataFrame,
 'social_security': pseudopeople.exceptions.DataSourceError,
 'taxes_1040': pandas.core.frame.DataFrame,
 'taxes_w2_and_1099': pandas.core.frame.DataFrame,
 'women_infants_and_children': pandas.core.frame.DataFrame}

In [43]:
sum(sizemb(v) for v in ri_data.values()) # 3 GB for one year of all the RI data

3019.5900990000005

# Check whether the exception classes have been fixed yet

Nope, they still derive directly from `BaseException` instead of `Exception`, so I had to add the specific exception types to my `try/except` block in `generate_datasets`.

In [44]:
# Oops, isinstance is for instantiated objects, not classes
isinstance(psp.exceptions.DataSourceError, BaseException)

False

In [47]:
psp.exceptions.DataSourceError.__base__

BaseException

In [48]:
issubclass(psp.exceptions.DataSourceError, BaseException)

True

In [68]:
issubclass(psp.exceptions.DataSourceError, Exception)

False

# See how many rows are in each dataset

In [57]:
{name: f'{len(df)=}' for name, df in ri_data.items() if isinstance(df, pd.DataFrame)}

{'american_community_survey': 'len(df)=1133',
 'current_population_survey': 'len(df)=4655',
 'decennial_census': 'len(df)=1073246',
 'taxes_1040': 'len(df)=673212',
 'taxes_w2_and_1099': 'len(df)=1051744',
 'women_infants_and_children': 'len(df)=14254'}

# See what columns and dtypes are in each dataset

In [50]:
{name: df.dtypes for name, df in ri_data.items() if isinstance(df, pd.DataFrame)}

{'american_community_survey': simulant_id                                 object
 household_id                                object
 survey_date                         datetime64[ns]
 first_name                                  object
 middle_initial                              object
 last_name                                   object
 age                                         object
 date_of_birth                               object
 street_number                               object
 street_name                                 object
 unit_number                                 object
 city                                        object
 state                                     category
 zipcode                                     object
 housing_type                                object
 relationship_to_reference_person          category
 sex                                       category
 race_ethnicity                            category
 dtype: object,
 'current_populatio

# Look at some of the RI datasets

In [51]:
ri_data.decennial_census

,simulant_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year
0,1007_574,1007_258,Harry,A,Cook,76,11/05/1943,2216,ronnie rd,NaN,east providence,RI,02908,Standard,Reference person,Male,Black,2020
1,1007_709,1007_322,Tonya,J,Troyer,32,01/03/1988,6615,spruce,NaN,portsmouth,RI,02812,Standard,Reference person,Female,White,2020
2,1007_710,1007_322,Daniel,I,Troyer,32,02/06/1988,6615,spruce,NaN,portsmouth,RI,02812,Standard,Sibling,Male,White,2020
3,1007_711,1007_322,Francisco,W,NaN,31,12/27/1988,6613,spruce,NaN,portsmouth,RI,02812,Standard,Sibling,Male,White,2020
4,1007_1281,1007_547,Christina,J,Smith,44,07/17/1975,60,putters pl,NaN,pawtucket,RI,02861,Standard,Reference person,Female,White,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073241,9971_1022747,9971_417220,Terry,M,Carrasco,64,10/28/1955,7597,noah road,NaN,warwick,RI,02908,Standard,Reference person,Male,Latino,2020
1073242,9971_1022748,9971_417220,Madison,K,Carrasco,17,08/17/2002,7597,noah road,NaN,warwick,RI,02908,Standard,Other relative,Female,Latino,2020
1073243,9971_1022756,9971_417225,Kyle,J,Lembke,32,08/14/1987,47,palmer square west,NaN,westerly,RI,02915,Standard,Reference person,Male,White,2020
1073244,9971_1022757,9971_417225,Erin,L,Lembke,37,04/20/1983,47,palmer square west,NaN,wsterly,RI,02915,Standard,Opp-sex spouse,Female,White,2020


In [52]:
ri_data.decennial_census.memory_usage(deep=True)

Index                                    128
simulant_id                         72774709
household_id                        72326211
first_name                          67173092
middle_initial                      61990529
last_name                           67810291
age                                 39173151
date_of_birth                       71542495
street_number                       63511552
street_name                         73867177
unit_number                         36291766
city                                71226252
state                                1078359
zipcode                             66199505
housing_type                        69789824
relationship_to_reference_person     1075149
sex                                  1073478
race_ethnicity                       1073995
year                                38636856
dtype: int64

In [53]:
sizemb(ri_data.decennial_census) # 876.6 MB for one decennial census of RI

876.614535

In [54]:
ri_data.taxes_1040

,simulant_id,household_id,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,...,dependent_2_first_name,dependent_2_last_name,dependent_2_ssn,dependent_3_first_name,dependent_3_last_name,dependent_3_ssn,dependent_4_first_name,dependent_4_last_name,dependent_4_ssn,tax_year
0,1007_573,1007_258,Judy,M,Cook,2216,ronnie rd,NaN,NaN,east providence,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
1,1007_709,1007_322,Tonya,J,Troyer,6615,spruce,NaN,NaN,portsmouth,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
2,1007_1683,1007_714,Mark,W,Gibbons,6217,wilson blvd,NaN,NaN,newport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,1007_2467,1007_1039,Stephen,J,Johnson,NaN,superior dr,NaN,NaN,east providence,...,Alex,Johnson,500-49-6133,Jasmin,Johnson,872-28-8355,NaN,NaN,NaN,2020
4,1007_3529,1007_424021,Lauren,C,Brooks,241,richard road,NaN,NaN,coventry,...,Alondra,Brooks,542-64-5002,NaN,NaN,NaN,NaN,NaN,NaN,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673207,9971_1035556,9971_282735,Sarah,H,Cornman,817,109th pl ne,NaN,NaN,burrillville,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
673208,9971_1035764,9971_384222,Roman,C,Farr,228,1st ave,NaN,NaN,north providence,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
673209,9971_1036655,9971_176574,Peyton,Z,Shaw,399,confederate point rd,NaN,NaN,pawtucket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
673210,9971_1038066,9971_425982,Elijah,J,Gonzalez-Gallegos,5037,west sunrise court,NaN,NaN,warwick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [55]:
ri_data.taxes_1040.memory_usage(deep=True)

Index                                 128
simulant_id                      45663764
household_id                     45326840
first_name                       42138169
middle_initial                   38872823
last_name                        42601432
mailing_address_street_number    39793573
mailing_address_street_name      46048020
mailing_address_unit_number      23251357
mailing_address_po_box           21562414
mailing_address_city             44715305
mailing_address_state              678325
mailing_address_zipcode          41537833
ssn                              45538791
spouse_first_name                23533092
spouse_middle_initial            23216950
spouse_last_name                 23569129
spouse_ssn                       23861327
dependent_1_first_name           25334509
dependent_1_last_name            25431457
dependent_1_ssn                  25963239
dependent_2_first_name           23492007
dependent_2_last_name            23537615
dependent_2_ssn                  2

In [56]:
sizemb(ri_data.taxes_1040) # 875.7 MB for one year of RI 1040 data

875.70045

# See how big a single shard of taxes 1040 data is in memory

In [58]:
%%time
ri_t1040_1007 = pd.read_parquet(ri_data_dir / 'taxes_1040/taxes_1040_1007.parquet')
ri_t1040_1007

CPU times: user 8.47 s, sys: 1.05 s, total: 9.52 s
Wall time: 9.48 s


,simulant_id,household_id,first_name_id,middle_name_id,last_name_id,age,copy_age,date_of_birth,copy_date_of_birth,sex,...,dependent_3_last_name,dependent_4_last_name,dependent_1_ssn,dependent_2_ssn,dependent_3_ssn,dependent_4_ssn,dependent_1_copy_ssn,dependent_2_copy_ssn,dependent_3_copy_ssn,dependent_4_copy_ssn
177,1007_573,1007_258,1007_573,1007_573,1007_573,72,NaN,1946-08-24,NaT,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,1007_1683,1007_714,1007_1683,1007_1683,1007_1683,53,NaN,1965-06-09,NaT,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051,1007_3734,1007_1564,1007_3734,1007_3734,1007_3734,34,NaN,1984-07-01,NaT,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1490,1007_5451,1007_2240,1007_5451,1007_5451,1007_5450,59,NaN,1959-04-15,NaT,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2172,1007_8066,1007_3290,1007_8066,1007_8066,1007_8066,38,4,1980-09-14,2011-12-12,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17230342,1007_1391609,1007_27029,1007_1391609,1007_1391609,1007_66633,0,32,2041-04-19,2008-05-18,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17230467,1007_1391787,1007_833625,1007_1391787,1007_1391787,1007_377544,0,36,2041-04-04,2004-08-01,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17230720,1007_1392196,1007_788811,1007_1392196,1007_1392196,1007_974070,0,41,2041-03-26,1999-07-15,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17230833,1007_1392373,1007_688108,1007_1392373,1007_1392373,1007_1248404,0,33,2041-03-27,2003-12-05,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# 1.4 GB for one shard, vs. 875 GB for the entire dataset after noising?
# Oh, right. The input data contains 20 years' worth of data, whereas I only requested one year's worth.
# And it looks like the input data contains a lot more columns than the final version.
sizemb(ri_t1040_1007)

1474.359795

# Look at what's in the raw 1040 data

In [60]:
ri_t1040_1007.dtypes

simulant_id             category
household_id            category
first_name_id           category
middle_name_id          category
last_name_id            category
                          ...   
dependent_4_ssn         category
dependent_1_copy_ssn    category
dependent_2_copy_ssn    category
dependent_3_copy_ssn    category
dependent_4_copy_ssn    category
Length: 95, dtype: object

In [61]:
ri_t1040_1007.memory_usage(deep=True)

Index                      412504
simulant_id             128307541
household_id             90590932
first_name_id           128307541
middle_name_id          128307541
                          ...    
dependent_4_ssn           3110922
dependent_1_copy_ssn     60623632
dependent_2_copy_ssn     35749392
dependent_3_copy_ssn     16751292
dependent_4_copy_ssn      6821632
Length: 96, dtype: int64

In [62]:
ri_t1040_1007.columns

Index(['simulant_id', 'household_id', 'first_name_id', 'middle_name_id',
       'last_name_id', 'age', 'copy_age', 'date_of_birth',
       'copy_date_of_birth', 'sex', 'has_ssn', 'copy_ssn', 'address_id',
       'po_box', 'state_id', 'puma', 'race_ethnicity',
       'relationship_to_reference_person', 'housing_type', 'tax_year', 'alive',
       'in_united_states', 'joint_filer', 'random_seed', 'year_of_birth',
       'state', 'ssn', 'first_name', 'middle_initial', 'last_name',
       'mailing_address_street_number', 'mailing_address_street_name',
       'mailing_address_unit_number', 'mailing_address_po_box',
       'mailing_address_state', 'mailing_address_city',
       'mailing_address_zipcode', 'spouse_simulant_id', 'spouse_household_id',
       'spouse_first_name_id', 'spouse_middle_name_id', 'spouse_last_name_id',
       'spouse_age', 'spouse_copy_age', 'spouse_date_of_birth',
       'spouse_copy_date_of_birth', 'spouse_sex', 'spouse_has_ssn',
       'spouse_copy_ssn', 'spouse_add

In [64]:
ri_t1040_1007.spouse_random_seed.nunique(dropna=False)

2

In [65]:
ri_t1040_1007.spouse_random_seed.value_counts()

1007    2569
Name: spouse_random_seed, dtype: int64

In [66]:
ri_t1040_1007.filter(like='state')

,state_id,in_united_states,state,mailing_address_state,spouse_state_id,spouse_in_united_states,spouse_state,spouse_mailing_address_state
177,44,True,RI,RI,NaN,None,NaN,NaN
473,44,True,RI,RI,NaN,None,NaN,NaN
1051,44,True,RI,RI,NaN,None,NaN,NaN
1490,44,True,RI,RI,NaN,None,NaN,NaN
2172,44,True,RI,RI,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...
17230342,44,True,RI,RI,NaN,None,NaN,NaN
17230467,44,True,RI,RI,NaN,None,NaN,NaN
17230720,44,True,RI,RI,NaN,None,NaN,NaN
17230833,44,True,RI,RI,NaN,None,NaN,NaN


In [67]:
51563*334 # Suggests 17.2 million rows total

17222042

# Try generating datasets while logging to file

In [94]:
%%time
with CodeTimer("Sample data generation", unit='s', logger_func=file_logger.info):
    data = alpha.generate_datasets(code_timer_unit='s', code_timer_logger_func=file_logger.info)
# file_logger.info(f"Generating sample datasets took {sample_timer.took} s")
data.keylist()

2023-08-23 17:04:00,995 - __main__ - INFO - Code block 'generate_american_community_survey' took: 0.90978 s
2023-08-23 17:04:01,760 - __main__ - INFO - Code block 'generate_current_population_survey' took: 0.76477 s
2023-08-23 17:04:03,943 - __main__ - INFO - Code block 'generate_decennial_census' took: 2.18201 s   
2023-08-23 17:04:05,562 - __main__ - INFO - Code block 'generate_social_security' took: 1.61837 s    
2023-08-23 17:04:10,544 - __main__ - INFO - Code block 'generate_taxes_1040' took: 4.98136 s         
2023-08-23 17:04:14,453 - __main__ - INFO - Code block 'generate_taxes_w2_and_1099' took: 3.90881 s  
2023-08-23 17:04:15,338 - __main__ - INFO - Code block 'generate_women_infants_and_children' took: 0.88387 s
2023-08-23 17:04:15,353 - __main__ - INFO - Code block 'Sample data generation' took: 15.26788 s


CPU times: user 15 s, sys: 150 ms, total: 15.2 s
Wall time: 15.3 s


['american_community_survey',
 'current_population_survey',
 'decennial_census',
 'social_security',
 'taxes_1040',
 'taxes_w2_and_1099',
 'women_infants_and_children']

In [92]:
!ls -halt *.log

-rw-rw-r-- 1 ndbs IHME-users 812 Aug 23 16:35 data_generation.log


In [95]:
!head -20 data_generation.log

2023-08-23 16:35:13,518 - __main__ - INFO - Code block 'generate_american_community_survey' took: 0.90410 s
2023-08-23 16:35:14,251 - __main__ - INFO - Code block 'generate_current_population_survey' took: 0.73190 s
2023-08-23 16:35:16,396 - __main__ - INFO - Code block 'generate_decennial_census' took: 2.14440 s
2023-08-23 16:35:18,002 - __main__ - INFO - Code block 'generate_social_security' took: 1.60542 s
2023-08-23 16:35:22,680 - __main__ - INFO - Code block 'generate_taxes_1040' took: 4.67757 s
2023-08-23 16:35:26,571 - __main__ - INFO - Code block 'generate_taxes_w2_and_1099' took: 3.89013 s
2023-08-23 16:35:27,453 - __main__ - INFO - Code block 'generate_women_infants_and_children' took: 0.88141 s
2023-08-23 16:35:27,470 - __main__ - INFO - Generating sample datasets took 14.856410640990362 s
2023-08-23 17:04:00,995 - __main__ - INFO - Code block 'generate_american_community_survey' took: 0.90978 s
2023-08-23 17:04:01,760 - __main__ - INFO - Code block 'generate_current_populat

In [82]:
!rm data_generation.log

# Generate RI data again but log times to file

In [96]:
with CodeTimer("RI data generation", logger_func=file_logger.info):
    data = alpha.generate_datasets(str(ri_data_dir), code_timer_logger_func=file_logger.info)

Applying noise:   0%|                                                       | 0/14 [00:00<?, ?type/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)


Applying noise:  93%|██████████████████████████████████████████▋   | 13/14 [00:00<00:00, 13.99type/s]
                                                                                                     IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)


Applying noise: 100%|█████████████████████████████████████

# How long did it take to generate all RI datasets?

Oops, I forgot to specify unit = minutes for the outer CodeTimer, so it reported the total time in miliseconds; conversion to minutes and hours is below.

According to the log file, it took 20_427_353 ms = 340.5 m = 5.67 h. Moreover the taxes 1040 dataset alone took 221.4 m = 3.69 h, or 65% of the total time. That means the rest of the datasets together took 119.1 m = 1.98 h.

In [97]:
# miliseconds -> minutes
20427353.84293 / 1000 / 60 # 340.5 minutes

340.45589738216665

In [98]:
# minutes -> hours
340.45589738216665 / 60 # 5.6 hours

5.674264956369444

In [99]:
221.38654 / 340.45589738216665

0.6502649585520042

In [100]:
(1-0.6502649585520042) * 340.45589738216665

119.06935738216667

In [101]:
119.06935738216667 / 60

1.984489289702778

In [102]:
221.38654 / 60

3.689775666666667

In [103]:
!tail -20 data_generation.log

2023-08-23 16:35:22,680 - __main__ - INFO - Code block 'generate_taxes_1040' took: 4.67757 s
2023-08-23 16:35:26,571 - __main__ - INFO - Code block 'generate_taxes_w2_and_1099' took: 3.89013 s
2023-08-23 16:35:27,453 - __main__ - INFO - Code block 'generate_women_infants_and_children' took: 0.88141 s
2023-08-23 16:35:27,470 - __main__ - INFO - Generating sample datasets took 14.856410640990362 s
2023-08-23 17:04:00,995 - __main__ - INFO - Code block 'generate_american_community_survey' took: 0.90978 s
2023-08-23 17:04:01,760 - __main__ - INFO - Code block 'generate_current_population_survey' took: 0.76477 s
2023-08-23 17:04:03,943 - __main__ - INFO - Code block 'generate_decennial_census' took: 2.18201 s
2023-08-23 17:04:05,562 - __main__ - INFO - Code block 'generate_social_security' took: 1.61837 s
2023-08-23 17:04:10,544 - __main__ - INFO - Code block 'generate_taxes_1040' took: 4.98136 s
2023-08-23 17:04:14,453 - __main__ - INFO - Code block 'generate_taxes_w2_and_1099' took: 3.908